# Urban scaling and coronavirus #

This notebook is a replication attempt of Stier et al.'s preprint on Arxiv [1]. This notebook brings together MSA definitions and census data to allow demographic calculations for MSAs in relation to the coronavirus outbreak. The MSAs are a county-level unit delineated by the Census Bureau (see https://www.census.gov/programs-surveys/metro-micro/about/delineation-files.html). The coronavirus outbreak data are provided by USAFacts (https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/, last downloaded March 25th, 2020 14:34 EDT).

.. [1] Andrew J Stier, Marc G. Berman, and Luis M. A. Bettencourt. March 23rd, 2020. COVID-19 attack rate increases with city size. arXiv:2003.10376v1 [q-bio.PE] https://arxiv.org/abs/2003.10376v1


In [1]:
#import key packages
import numpy as np
import pandas
import datetime #the covid dataset uses datetimes as column indices
import scipy.stats
import matplotlib.pyplot as plt
import ipywidgets as widgets

#Load the datasets: the MSA delineations, the ACS 2018 population estimates, and the USAFacts Coronavirus dataset
msas = pandas.read_excel(r'data/USCensus/Delineations/list1_Sep_2018.xls',header=2)
pop = pandas.read_excel(r'data/USCensus/Population/co-est2018-alldata.xls')
covid = pandas.read_excel(r'data/Coronavirus/USAFacts/covid_confirmed_usafacts.xls')
deaths = pandas.read_excel(r'data/Coronavirus/USAFacts/covid_deaths_usafacts.xls')

Now that the data are loaded, pick dates to run the COVID-19 regression on. (This is a linear regression of normalized, natural log active case counts (i.e., confirmed tests with deaths removed). These assumptions are all replicating reference [1], above.

Note: this takes a few minutes to run! Set output to True to see all the places that can't be included.

In [2]:
#Assemble the state, metro/micro areas, pop data, and covid cases
dateStart = widgets.DatePicker(description = 'Start date')
dateStart.value = datetime.datetime(2020, 3, 13,0,0)
dateEnd = widgets.DatePicker(description = 'End date')
dateEnd.value = datetime.date(2020, 3, 19) #INCLUSIVE; note this date was chosen for comparability with the arxiv preprint

def coallate_analyze(dateStart,dateEnd, msas, pop, covid, deaths, output=False):
    """Utility function for ipywidgets.
    
    This function takes as input the US Census MSA definitions and 2018 ACS data, the USAFacts
    coronavirus case and death counts at the county level, and the start and end dates for the 
    statistical regression in keeping with the methodology outlined in [1]. It tabulates these data
    and outputs a graph
    
    """
    global df #to remain accessible after the function is run
    dateStart = datetime.datetime.combine(dateStart,datetime.time())
    dateEnd = datetime.datetime.combine(dateEnd,datetime.time())
    data = [] #to store the actual data to be exported and used
    popnotfound = [] #to store fips of counties that can't be found
    covidnotfound = [] #
    covidnotincl = []
    days = (dateEnd - dateStart).days+1
    col_names = ['CBSA','Title','MetroMicro','Pop2018','COVIDEnd','AttackRate','r']
    #Leaving out Puerto Rico because it is not in the census data; I'm sorry! I hope to remedy this.
    for cbsa in msas.loc[(msas['FIPS State Code'] != 72)]['CBSA Code'][:-4].unique():
        #Get the MSA information
        #cbsa = '10740' #ABQ metro CBSA code for testing
        counties = msas.loc[msas['CBSA Code'] == cbsa]
        row = [cbsa,counties.loc[counties.index[0]]['CBSA Title'],counties.loc[counties.index[0]]['Metropolitan/Micropolitan Statistical Area']]
        #for all state and county codes, go through and select the relevant pop data
        pop_total = 0 #to sum the population
        covid_last = 0 #the total number of cases in the last day of the series
        covid_series = [0]*days #This stores just cases, not people who died
        #Loop through every constituent county to get the population as well as the COVID cases
        for s, c in zip(counties['FIPS State Code'],counties['FIPS County Code']):
            fips = int(s*1000 + c) #str(int(s)) + '0'*(3-len(str(int(c))))+str(int(c))
            if any((pop.STATE == int(s)) & (pop.COUNTY == int(c))):
                pop_total += int(pop.loc[(pop.STATE == int(s)) & (pop.COUNTY == int(c))]['POPESTIMATE2018'])
            else: #If the fips code doesn't exist, skip this county but record why.
                if output == True:
                    print(str(fips) + ' was not found in the ACS data.')
                popnotfound.append(fips)
            if any(fips == covid.countyFIPS):
                #This data sometimes contains duplicates; it is assumed that together these are the true total.
                covid_last += int(sum(covid.loc[(covid.countyFIPS == fips)][dateEnd]))-int(sum(deaths.loc[(deaths.countyFIPS == fips)][dateEnd]))
                for i,d in zip(range(days),pandas.date_range(dateStart,dateEnd)):
                    covid_series[i] += int(sum(covid.loc[(covid.countyFIPS == fips)][d])) - int(sum(deaths.loc[(deaths.countyFIPS == fips)][d]))
            else:
                if output == True:
                    print(str(fips) + ' was not found in the COVID data.')
                covidnotfound.append(fips)
        row.append(pop_total) 
        row.append(covid_last)
        #Now calculate the r
        if any(pandas.isna(covid_series)) or (covid_series[-1] <= 3) or any([cs <= 0 for cs in covid_series]):
            if output == True:
                print(row[1] + ' had too few cases for inclusion')
            covidnotincl.append(cbsa)
            row.append(np.nan)
            row.append(np.nan)
        else:
            #normalize the covid_series so that the March 13th data is 1
            covid_series = [cs * 1. / covid_series[0] for cs in covid_series]
            #run a regression
            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(range(days),[np.log(cs) for cs in covid_series])
            row.append(slope)
            row.append(r_value)
        data.append(row)
    df = pandas.DataFrame(data,columns=col_names)
    print('\n \n \n Data are gathered for cases from ' + dateStart.strftime("%B %d, %Y") +' to ' + dateEnd.strftime("%B %d, %Y") + '! Run the next box when ready.')

#Run with some interactability for the dates
print('Change the dates to your window of interest, then click "Run Interact" to run the regression.')
widgets.interact_manual(coallate_analyze, dateStart = dateStart, dateEnd = dateEnd, msas = widgets.fixed(msas), pop = widgets.fixed(pop), covid = widgets.fixed(covid), deaths = widgets.fixed(deaths),output = False)
print('')

Change the dates to your window of interest, then click "Run Interact" to run the regression.


interactive(children=(DatePicker(value=datetime.datetime(2020, 3, 13, 0, 0), description='Start date'), DatePi…

In [3]:
def plot_some_figures(df):
    """
    Make plots.
    """
    ####################################################
    #Now that the data are collated, generate the graphs
    #Plot the chart from figure 1a from the arxiv preprint
    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2)    
    fig.set_figheight(12)
    fig.set_figwidth(12)
    points, = ax1.plot([np.log(x) for x in df.Pop2018],[np.log(y) for y in df.AttackRate],'bo')
    #Now run a linear regression to calculate any potential power law behavior
    #Only include those where an attack rate could be esitmated and its a Metropolitan Area (not Micro)
    which = df.index[(pandas.isna(df.AttackRate)==False) & (df.MetroMicro == 'Metropolitan Statistical Area')] 
    global slope, intercept
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress([np.log(x) for x in df.Pop2018[which]],[np.log(y) for y in df.AttackRate[which]])
    line, = ax1.plot([np.log(x) for x in df.Pop2018],[np.log(x)*slope + intercept for x in df.Pop2018],'k-')
    ax1.set_xlabel('log(MSA Population)')
    ax1.set_ylabel('log(Estimated attack rate)')
    ax1.set_title('Attack rate vs. pop (r = {:0.3f}, p = {:0.3f}, exponent {:1.4f})'.format(r_value,p_value,slope))
    # Look at how well correlation describes the growth curves for the selected dates
    ax2.hist(df.r[which][pandas.isna(df.r[which]) == False],bins=np.arange(0.0,1.10,.10))
    ax2.set_title('Correlations for city-by-city exponential growth')
    # Make a q-q plot
    #Calculate teh residual distribution of the data
    residuals = [np.log(y) - (intercept + slope*np.log(x)) for x,y in zip( df.Pop2018[which], df.AttackRate[which])]
    (osm, osr), (qqslope, qqintercept, qqr) = scipy.stats.probplot(residuals,fit=True, plot=ax3)    
    ax3.set_title('Q-Q plot against normal (slope = %f, r = %f)' % (qqslope, qqr))
    plt.show
    
widgets.interact(plot_some_figures,df = widgets.fixed(df))
print('')

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [4]:
def lookup_a_city(df,pickacity):
    w = df.index[df.Title == pickacity]
    if len(w) > 1:
        print('Uh-oh, more than one city by that name!')
    else:
        w = df.index[df.Title == cityselector.value][0]
        print(str(df.Title[w]))
        print(str(df.MetroMicro[w]))
        print('Population (2018 ACS estimate): %d ' % (df.Pop2018[w]))
        print('Covid cases by ' + dateEnd.value.strftime("%B %d, %Y") + ': ' + str(df.COVIDEnd[w]))
        if pandas.isna(df.AttackRate[w]):
            print('There was not sufficient data (or another error occurred) to estimate a growth rate')
        else:
            print('COVID-19 attack rate (from regression): %f' % (df.AttackRate[w]))
            print('Correlation for that regression: %f' % (df.r[w]))
            print('Subsequent R: %f' % (df.r[w]*4.5 + 1))
            print('Residual for the power-law regression: %f' % (np.log(df.AttackRate[w]) - (intercept + slope*np.log(df.Pop2018[w]))))

cityselector = widgets.Select(options = df.Title[df.MetroMicro == 'Metropolitan Statistical Area'],description = 'Pick a city to examine specifics')
widgets.interact(lookup_a_city,df = widgets.fixed(df),pickacity = cityselector, output = False)
print('')

interactive(children=(Select(description='Pick a city to examine specifics', options=('Abilene, TX', 'Akron, O…